In [1]:
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import numpy as np
import cv2

In [2]:
def calibrate_camera(object_point_arr, image_point_arr):
    object_point_arr = np.asarray(object_point_arr)
    image_point_arr = np.asarray(image_point_arr)
    
    num_points = object_point_arr.shape[0]
    T_object, norm_object = normalize_points(3, object_point_arr)
    T_img, norm_img = normalize_points(2, image_point_arr)
    A = []
    for i in range(num_points):
        x, y, z = norm_object[i, 0], norm_object[i, 1], norm_object[i, 2]
        u, v = norm_img[i, 0], norm_img[i, 1]
        A.append( [x, y, z, 1, 0, 0, 0, 0, -u*x, -u*y, -u*z, -u] )
        A.append( [0, 0, 0, 0, x, y, z, 1, -v*x, -v*y, -v*z, -v] )
#     print(A.shape)
    A = np.asarray(A)
    #print(A)
    U, S, Vh = np.linalg.svd(A)
    L = Vh[-1,:]/Vh[-1,-1]
    H = L.reshape(3, 4)
    H = np.dot(np.dot(np.linalg.pinv(T_img), H), T_object)
    H = H/H[-1, -1]
    L = H.flatten(0)
    uv2 = np.dot(H, np.concatenate((object_point_arr.T, np.ones((1, object_point_arr.shape[0])))))
    uv2 = uv2/uv2[2,:]
    err = np.sqrt(np.mean(np.sum((uv2[0:2,:].T - image_point_arr))))
    return L, err
        
        

In [3]:
def normalize_points(input_dimension, point_arr):
    point_arr = np.asarray(point_arr)
    mean, standard_deviation = np.mean(point_arr, 0), np.std(point_arr)
#     print(mean)
    if (input_dimension == 2):
        transformation_matrix = np.array([[standard_deviation, 0, mean[0]], [0, standard_deviation, mean[1]], [0, 0, 1]])
    elif (input_dimension == 3):
        transformation_matrix = np.array([[standard_deviation, 0, 0, mean[0]], [0, standard_deviation, 0, mean[1]], [0, 0, standard_deviation, mean[2]], [0, 0, 0, 1]])
    transformation_matrix = np.linalg.inv(transformation_matrix)
    transformed_data = np.dot(transformation_matrix, np.concatenate((point_arr.T, np.ones((1, point_arr.shape[0])))))
    transformed_data = transformed_data[0:input_dimension,:].T
    return transformation_matrix, transformed_data    

In [4]:
real_obj1 = [[500,0,0], [540,0,0], [600,0,0], [560,0,0], [660,0,0], [500,40,0], [560,40,0], [600,40,0]]
real_obj2 = [[500,0,200], [540,0,200], [600,0,200], [560,0,200], [660,0,200], [500,40,200], [560,40,200], [600,40,200]]
real_obj = real_obj1 + real_obj2

In [5]:
# real_obj

In [6]:
view1 = [[131,35],[164,36],[215,37],[184,37],[263,39],[131,69],[183,70],[214,70]]
view2 = [[95,31],[122,32],[164,33],[138,32],[220,35],[95,58],[138,59],[163,60]]
view = view1 + view2

In [7]:
param, err = calibrate_camera(real_obj, view)
print(param)
# param1, err1 = calibrate_camera(real_obj1, view1)
# print(param1)
H1 = param[0:9]
H2 = param[9:]

[ 6.39798582e-01  1.39854756e-02 -1.30914954e-01 -2.06577305e+02
  7.51156032e-03  6.76412266e-01 -1.21955755e-02  2.69107443e+01
 -2.89491588e-04  1.74523408e-04  3.69860695e-04  1.00000000e+00]


/home/stark/.local/lib/python3.5/site-packages/ipykernel_launcher.py:22: DeprecationWarning: Non-string object detected for the array ordering. Please pass in 'C', 'F', 'A', or 'K' instead


In [8]:
print(err)

0.19247873547501718


In [9]:
param = param.reshape(3,4)

In [10]:
param = np.asarray(param)

In [11]:
param

array([[ 6.39798582e-01,  1.39854756e-02, -1.30914954e-01,
        -2.06577305e+02],
       [ 7.51156032e-03,  6.76412266e-01, -1.21955755e-02,
         2.69107443e+01],
       [-2.89491588e-04,  1.74523408e-04,  3.69860695e-04,
         1.00000000e+00]])

In [12]:
np.linalg.pinv(param)

array([[ 1.71338107e+00, -1.31795926e-01,  3.57492403e+02],
       [-4.41457703e-02,  1.49060584e+00, -4.92328163e+01],
       [-1.55099126e-01, -3.40143012e-02, -3.11242638e+01],
       [ 5.61078949e-04, -2.85718870e-04,  1.12359496e+00]])

In [13]:
H1 = H1.reshape(3,3)
H2 = H2.reshape(3,1)

In [14]:
H1

array([[ 6.39798582e-01,  1.39854756e-02, -1.30914954e-01],
       [-2.06577305e+02,  7.51156032e-03,  6.76412266e-01],
       [-1.21955755e-02,  2.69107443e+01, -2.89491588e-04]])

In [15]:
H2

array([[1.74523408e-04],
       [3.69860695e-04],
       [1.00000000e+00]])